### Penjelasan

<p> Sebuah perusahaan e-commerce sedang mengembangkan websitenya dengan membuat halaman landing page baru untuk meningkatkan
"converting" pada pengguna. Converted disini mengindikasikan bahwa pengguna akan membeli atau tidak produk pada e-commerce tersebut.
Converted bernilai 1 jika pengguna membeli produk pada e-commerce dan bernilai 0 jika tidak membeli.

### Import libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Import Dataset yang akan digunakan

In [17]:
file = pd.read_csv("ab_data.csv")

In [18]:
file.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


### Data Cleansing

Periksa jika ada cell yang masih bernilai null atau tidak

In [19]:
file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
user_id         294478 non-null int64
timestamp       294478 non-null object
group           294478 non-null object
landing_page    294478 non-null object
converted       294478 non-null int64
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


Tidak ada yang bernilai null

In [42]:
file3 = file
file3 = file[((file.group=='treatment') & (file.landing_page=='old_page')) | ((file.group=='control') & (file.landing_page=='new_page'))]

file3.head()

,user_id,timestamp,group,landing_page,converted
22,767017,2017-01-12 22:58:14.991443,control,new_page,0
240,733976,2017-01-11 15:11:16.407599,control,new_page,0
308,857184,2017-01-20 07:34:59.832626,treatment,old_page,0
327,686623,2017-01-09 14:26:40.734775,treatment,old_page,0
357,856078,2017-01-12 12:29:30.354835,treatment,old_page,0


In [44]:
str(len(file3))

'3893'

Terdapat kesalahan pada penginputan data dimana seharusnya pada perlakuan control akan mendapakan landing page lama dan perlakuan treatment mendapatkan landing page baru

Karena terdapat sedikit kesalahan pada penginputan data, maka diperlukan penyaringan data kembali. Ambil data yang dimana ketika kolom grupnya adalah "treatment" maka kolom landing pagenya adalah "new_page" dan
juga ketika kolom grupnya adalah "control" maka landing pagenya adalah "old_page"

In [20]:
file2 = file
file2 = file[((file.group=='treatment') & (file.landing_page=='new_page')) | ((file.group=='control') & (file.landing_page=='old_page'))]

In [37]:
print(file2.head())
print('\n')
print('Rows of dataframe : ' + str(len(file2)) + ', dengan ' + str(len(file2.user_id.unique())) + ' pengguna')

   user_id                   timestamp      group landing_page  converted
0   851104  2017-01-21 22:11:48.556739    control     old_page          0
1   804228  2017-01-12 08:01:45.159739    control     old_page          0
2   661590  2017-01-11 16:55:06.154213  treatment     new_page          0
3   853541  2017-01-08 18:28:03.143765  treatment     new_page          0
4   864975  2017-01-21 01:52:26.210827    control     old_page          1


Rows of dataframe : 290585, dengan 290584 pengguna


Terdapat satu data dimana user_id nya terduplikasi atau terinput dalam data sebagai peserta eksperimen sebanyak dua kali.
Kemudian, salah satu data yang terduplikasi tersebut akan dihapus

In [23]:
file2.user_id[file2.user_id.duplicated()]

2893    773192
Name: user_id, dtype: int64

Periksa kembali apakah masih ada cell yang masih terduplikasi

In [24]:
file2 = file2.drop_duplicates()
sum(file2.duplicated())

0

In [55]:
ab_converted = file2[['group', 'converted']].groupby('group', as_index=False).converted.mean()
ab_converted.converted = round((ab_converted.converted)*100,2)
ab_converted.columns = ['Group', 'Persentase']
print('Persentase Untuk Control dan Treatment :')
print(ab_converted)

Persentase Untuk Control dan Treatment :
       Group  Persentase
0    control       12.04
1  treatment       11.88


Terdapat sedikit perbedaan persentase antara control dan treatment. Namun, perbedaan kecil ini dapat memberikan dampak yang cukup besar pada keputusan dalam AB Testing ini sehingga tetap diperlukan Uji Hipotesis

### Uji Hipotesis

In [30]:
def z_two_proportion(x1, x2, n1, n2) :
    "Calculate test statistic z value for Hypothesis Test Regarding Two Population Proportions"
    p = (x1+x2)/(n1+n2)
    p1 = x1/n1
    p2 = x2/n2
    y = p1 - p2
    q = np.sqrt(p*(1-p)*((1/n1)+(1/n2)))
    z = (p1-p2)/q
    return round (z,3)

<p>Dari data diatas dapat terlihat bahwa ada sedikit perbedaan persentase antara halaman web baru dan halaman web lama.
Namun, dalam pengujian A/B Testing perbedaan kecil antara dua perlakuan dapat memberikan dampak yang cukup signifikan. Oleh karena itu, selanjutnya akan dilakukan Uji Hipotesis sebagai berikut:

<p>Taraf nyata yang digunakan adalah α = 0.1 (10%)
<p>p1 = proporsi pengguna yang akhirnya memutuskan untuk membeli produk dengan tampilan web lama "old_page"
<p>p2 = proporsi pengguna yang akhirnya memutuskan untuk membeli produk dengan tampilan web baru "new_page"

<p>Hipotesis yang akan diuji

<p>H0 : p1 <= p2
<p>H1 : p1 > p2


In [31]:
old = len(file2.query('group=="control" and converted==1'))
new = len(file2.query('group=="treatment" and converted==1'))
n_old = len(file2.query('group=="control"'))
n_new = len(file2.query('group=="treatment"'))
z_retention_1 = z_two_proportion(old, new, n_old, n_new)
print('test statistic z :', z_retention_1)

test statistic z : 1.312


Dengan taraf nyata α = 0.1 (10%) dan z(0.1) = 1.280. Jadi, test statistik menunjukkan bahwa z = 1.312 > z(0.1) = 1.280
<p> Sehingga keputusannya adalah tolak H0 dan terima H1. Dapat disimpulkan bahwa tidak perlu adanya perubahan pada website
    halaman landing page dan tetap menggunakan halaman lama (old_page) pada landing page e-commerce